<a href="https://colab.research.google.com/github/RobsonPalerma/energy_consumption.csv/blob/main/Etapa_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) from google.colab import files

Importa o módulo files do Google Colab.

É ele que permite abrir a janelinha para enviar arquivos do seu computador para o notebook.

In [5]:
from google.colab import files
uploaded = files.upload()


Saving energy_consumption (4).csv to energy_consumption (4) (1).csv


Importa o Pandas, biblioteca para trabalhar com DataFrames.

In [7]:
# ==============================
# ETAPA 2: PRÉ-PROCESSAMENTO
# ==============================
import pandas as pd
import numpy as np

df = pd.read_csv('energy_consumption (4).csv')
df.head()


,house_id,house_area_sqm,num_residents,num_appliances,air_conditioning,heating_system,energy_efficiency_rating,avg_temperature,home_office_hours,has_pool,solar_panels,house_age_years,insulation_quality,num_rooms,electric_car,smart_thermostat,monthly_consumption_kwh
0,HOUSE02311,195.78,5,26.0,2,Electric,C,40.00,8.0,No,Yes,4.0,Good,4,No,Yes,595.84
1,HOUSE01565,109.37,4,13.0,NaN,NaN,B,8.68,0.0,No,No,33.0,Average,5,No,yes,328.93
2,HOUSE02471,158.97,1,7.0,2,Gas,C,21.72,0.0,No,No,NaN,Good,8,No,Yes,308.77
3,HOUSE02286,155.23,3,6.0,3+,Gas,B,10.66,8.0,No,No,5.0,Good,6,No,No,392.40
4,HOUSE00978,NaN,4,8.0,1,Gas,E,25.83,0.0,Yes,No,6.0,Poor,8,No,Yes,485.11


✔ Lê o arquivo enviado
✔ Interpreta como CSV
✔ Converte em DataFrame do Pandas

uploaded[file_name] → pega os bytes do arquivo
io.BytesIO(...) → transforma esses bytes em um arquivo "virtual"
pd.read_csv(...) → lê esse arquivo virtual como CSV

In [8]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2510 entries, 0 to 2509
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   house_id                  2510 non-null   object 
 1   house_area_sqm            2287 non-null   float64
 2   num_residents             2510 non-null   int64  
 3   num_appliances            2274 non-null   float64
 4   air_conditioning          1736 non-null   object 
 5   heating_system            2003 non-null   object 
 6   energy_efficiency_rating  2510 non-null   object 
 7   avg_temperature           2263 non-null   float64
 8   home_office_hours         2211 non-null   float64
 9   has_pool                  2510 non-null   object 
 10  solar_panels              2510 non-null   object 
 11  house_age_years           2363 non-null   float64
 12  insulation_quality        2223 non-null   object 
 13  num_rooms                 2510 non-null   int64  
 14  electric

Esse código serve para separar automaticamente as colunas numéricas e categóricas do seu DataFrame (df).
É muito usado na etapa de pré-processamento, porque números e categorias precisam de tratamentos diferentes.

In [9]:
num_cols = df.select_dtypes(include=['int64','float64']).columns
cat_cols = df.select_dtypes(include=['object']).columns

num_cols, cat_cols


(Index(['house_area_sqm', 'num_residents', 'num_appliances', 'avg_temperature',
        'home_office_hours', 'house_age_years', 'num_rooms',
        'monthly_consumption_kwh'],
       dtype='object'),
 Index(['house_id', 'air_conditioning', 'heating_system',
        'energy_efficiency_rating', 'has_pool', 'solar_panels',
        'insulation_quality', 'electric_car', 'smart_thermostat'],
       dtype='object'))

Aqui você cria um objeto que sabe como preencher valores faltantes.

strategy='median' → significa:

Se alguma coluna numérica tiver valores faltando, ele vai preencher usando a mediana da coluna.

📌 Por que mediana?
Porque é mais robusta para dados com outliers.

In [10]:
from sklearn.impute import SimpleImputer

imputer_num = SimpleImputer(strategy='median')
df[num_cols] = imputer_num.fit_transform(df[num_cols])



Essa linha cria um objeto que sabe como **preencher valores faltantes (NaN)**, mas agora **para colunas categóricas**.

- `strategy='most_frequent'` significa:
  👉 Preencher valores faltando usando o **valor mais comum da coluna** (a moda).

📌 Exemplo:  
Se a coluna "cidade" tem esses valores:



In [11]:
imputer_cat = SimpleImputer(strategy='most_frequent')
df[cat_cols] = imputer_cat.fit_transform(df[cat_cols])


✅ O que esse código faz?

Esse código remove outliers (valores muito fora do normal) nas colunas numéricas do DataFrame.

Ele usa o método IQR (Intervalo Interquartil), que é um dos mais usados em ciência de dados.

In [12]:
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    df[col] = np.where(df[col] < limite_inferior, limite_inferior, df[col])
    df[col] = np.where(df[col] > limite_superior, limite_superior, df[col])


O que o get_dummies faz?

Pega cada coluna categórica (as que estavam em cat_cols)

Cria novas colunas 0 ou 1 para cada categoria existente

Converte texto → números

In [13]:
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)
df.head()


,house_area_sqm,num_residents,num_appliances,avg_temperature,home_office_hours,house_age_years,num_rooms,monthly_consumption_kwh,house_id_HOUSE00002,house_id_HOUSE00003,...,insulation_quality_poor,electric_car_No,electric_car_Yes,electric_car_no,electric_car_yes,smart_thermostat_ Yes,smart_thermostat_No,smart_thermostat_Yes,smart_thermostat_no,smart_thermostat_yes
0,195.78,5.0,26.0,40.00,8.0,4.0,4.0,595.84,False,False,...,False,True,False,False,False,False,False,True,False,False
1,109.37,4.0,13.0,8.68,0.0,33.0,5.0,328.93,False,False,...,False,True,False,False,False,False,False,False,False,True
2,158.97,1.0,7.0,21.72,0.0,24.0,8.0,308.77,False,False,...,False,True,False,False,False,False,False,True,False,False
3,155.23,3.0,6.0,10.66,8.0,5.0,6.0,392.40,False,False,...,False,True,False,False,False,False,True,False,False,False
4,153.83,4.0,8.0,25.83,0.0,6.0,8.0,485.11,False,False,...,False,True,False,False,False,False,False,True,False,False


StandardScaler deixa todas as colunas numéricas na mesma escala (média 0, desvio 1).

fit_transform() aprende a média/desvio e aplica a normalização nos dados.

joblib.dump() serve para salvar o scaler, mas não pode ser salvo como CSV — deve ser salvo como .pkl.

In [15]:
from sklearn.preprocessing import StandardScaler
import joblib

scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

# Salvar scaler
joblib.dump(scaler, 'energy_consumption (4).csv')


['energy_consumption (4).csv']

In [16]:
# Exemplo caso existam colunas adequadas
# df['consumo_por_m2'] = df['monthly_consumption_kwh'] / df['house_area_sqm']


Salvando o DataFrame df em um arquivo CSV chamado energy_consumption (4).csv.

index=False significa que não vai salvar a coluna de índice do Pandas no arquivo.
(Ou seja, o CSV fica mais limpo, só com suas colunas reais.)

In [17]:
df.to_csv('energy_consumption (4).csv', index=False)


No print("Dimensões finais:", df.shape)

Mostra quantas linhas e colunas o DataFrame tem após o pré-processamento.

df.shape retorna uma tupla:
➝ (número de linhas, número de colunas)

Exemplo:
Se aparecer (2500, 32), significa:

2500 linhas
32 colunas
df.head()

Mostra as 5 primeiras linhas do DataFrame.

Serve para conferir rapidamente se o dataset ficou correto depois das transformações.

In [18]:
print("Dimensões finais:", df.shape)
df.head()


Dimensões finais: (2510, 2574)


,house_area_sqm,num_residents,num_appliances,avg_temperature,home_office_hours,house_age_years,num_rooms,monthly_consumption_kwh,house_id_HOUSE00002,house_id_HOUSE00003,...,insulation_quality_poor,electric_car_No,electric_car_Yes,electric_car_no,electric_car_yes,smart_thermostat_ Yes,smart_thermostat_No,smart_thermostat_Yes,smart_thermostat_no,smart_thermostat_yes
0,0.896033,1.591621,1.172042,2.379405,1.110131,-1.464831,-0.227641,2.038006,False,False,...,False,True,False,False,False,False,False,True,False,False
1,-0.934338,0.779305,-0.585482,-1.760621,-1.103297,0.576152,0.209527,-0.793171,False,False,...,False,True,False,False,False,False,False,False,False,True
2,0.116309,-1.657642,-1.396648,-0.036932,-1.103297,-0.057256,1.521030,-1.007013,False,False,...,False,True,False,False,False,False,False,True,False,False
3,0.037087,-0.033010,-1.531842,-1.498895,1.110131,-1.394452,0.646694,-0.119930,False,False,...,False,True,False,False,False,False,True,False,False,False
4,0.007431,0.779305,-1.261453,0.506347,-1.103297,-1.324073,1.521030,0.863467,False,False,...,False,True,False,False,False,False,False,True,False,False
